In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz #install Apache Spark
!tar xf spark-3.0.1-bin-hadoop2.7.tgz  

In [ ]:
!pip install pyspark==3.0.1

     |████████████████████████████████| 204.2MB 65kB/s 
     |████████████████████████████████| 204kB 22.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=e07efa961c6e5a90bde33eec43708d6dc4b1a0eeff98633040f5f23f1447d9cf
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
import os 
os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME']='/content/spark-3.0.1-bin-hadoop2.7'


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local[*]").appName("Task 1").getOrCreate()  # use all cores of your CPU - each core has thread

#**Challenge 1 -  PySpark and SQL Querying**

Write a PySpark code to read all the files in the `by-day` folder, and displays a table that shows the ID of the customer and the the total amount they paid for all their purchases. Make sure to drop any stock that has the word *WATER* in it before calculating the total price.

In [ ]:
df1 = spark.read.format('csv').option("header", "true").option("inferSchema","true").load("by-day/*.csv")

In [ ]:
df1.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    21544|SKULLS  WATER TRA...|      48|2011-12-05 08:38:00|     0.85|   14075.0|United Kingdom|
|   580538|    23126|FELTCRA

In [ ]:
from pyspark.sql.functions import col
df1 = df1.select(df1.columns).where(~col("Description").like("% WATER %"))
df1.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    23126|FELTCRAFT GIRL AM...|       8|2011-12-05 08:38:00|     4.95|   14075.0|United Kingdom|
|   580538|    21833|CAMOUFL

Solve the same question using raw SQL.

In [ ]:
from pyspark.sql.functions import desc
df2 = df1.selectExpr("CustomerID", "(UnitPrice * Quantity) as total_cost").groupby(col("CustomerID")).sum("total_cost").sort(desc("sum(total_cost)"))

In [ ]:
df2.show()

+----------+------------------+
|CustomerID|   sum(total_cost)|
+----------+------------------+
|      null|1369607.5800000506|
|   14646.0| 276588.9899999999|
|   18102.0|256438.48999999993|
|   17450.0|         179989.37|
|   14911.0|129805.12000000016|
|   12415.0|123556.25000000001|
|   14156.0|109706.65000000004|
|   17511.0| 86752.45999999999|
|   13694.0|           62653.1|
|   16684.0|62095.280000000006|
|   15311.0| 59394.58999999995|
|   14096.0| 56220.29999999983|
|   15061.0| 54228.73999999999|
|   13089.0| 53372.35999999998|
|   17949.0|          52750.84|
|   15769.0|          51731.92|
|   16029.0|          50992.61|
|   14298.0| 49660.00999999999|
|   14088.0|          49077.91|
|   17841.0| 39383.76999999996|
+----------+------------------+
only showing top 20 rows



In [ ]:
df1 = spark.read.format('csv').option("header", "true").option("inferSchema","true").load("by-day/*.csv")
df1.createOrReplaceTempView('my_retail')

In [ ]:
sqlQuery = spark.sql(""" SELECT CustomerID, sum(Quantity* UnitPrice) AS Total from my_retail WHERE Description NOT LIKE '% WATER %' GROUP BY CustomerID ORDER BY Total DESC """)

In [ ]:
sqlQuery.show()

+----------+------------------+
|CustomerID|             Total|
+----------+------------------+
|      null|1369607.5800000506|
|   14646.0| 276588.9899999999|
|   18102.0|256438.48999999993|
|   17450.0|         179989.37|
|   14911.0|129805.12000000016|
|   12415.0|123556.25000000001|
|   14156.0|109706.65000000004|
|   17511.0| 86752.45999999999|
|   13694.0|           62653.1|
|   16684.0|62095.280000000006|
|   15311.0| 59394.58999999995|
|   14096.0| 56220.29999999983|
|   15061.0| 54228.73999999999|
|   13089.0| 53372.35999999998|
|   17949.0|          52750.84|
|   15769.0|          51731.92|
|   16029.0|          50992.61|
|   14298.0| 49660.00999999999|
|   14088.0|          49077.91|
|   17841.0| 39383.76999999996|
+----------+------------------+
only showing top 20 rows



#**Challenge 2-  Pipeline**

Applying the following tasks to create your ML pipeline :

1. First, read all the files in the `by-day` folder.
2. Next, create a pipeline using the `Pipeline()` function, in which the indexer is applied followed by encoder and finally the assembler.
3. Afterwards, fit and transform the data at hand using the pipeline we created which includes: StringIndexer, OneHotEncoder, and VectorAssembler. 


In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
sindexer = StringIndexer().setInputCol("day_of_week").setOutputCol("day_of_week_indexed")
ohe = OneHotEncoder().setInputCol("day_of_week_indexed").setOutputCol("day_of_week_encoded")
va = VectorAssembler().setInputCols(["UnitPrice","Quantity"]).setOutputCol("Unit-Quantity")

In [ ]:
from pyspark.sql.functions import date_format,col
my_df = df1.withColumn("day_of_week",date_format(col("InvoiceDate"),"EEEE"))

In [ ]:
my_df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|     Monday|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|     Monday|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|     Monday|
|   580538|    21544|SKULLS  WATER TRA..

In [ ]:
from pyspark.ml import Pipeline
my_pl = Pipeline().setStages([sindexer,ohe,va])

In [ ]:
fittedpl = my_pl.fit(my_df)
transpl = fittedpl.transform(my_df)

In [ ]:
transpl.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-------------------+-------------------+-------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|day_of_week_indexed|day_of_week_encoded|Unit-Quantity|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-------------------+-------------------+-------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|     Monday|                2.0|      (5,[2],[1.0])|  [1.79,48.0]|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|                2.0|      (5,[2],[1.0])|  [1.25,20.0]|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|     Monday|           